# Mobile App for Lottery Addiction

In this project, I'll contribute to the development of a mobile app that is meant to help lottery addicts better estimate their chances of winning.

Many people start playing the lottery for fun, but for some this activity turns into a habit which eventually escalates into addiction. Like other compulsive gamblers, lottery addicts soon begin spending from their savings and loans, they start to accumulate debts, and eventually engage in desperate behaviors like theft.

A medical institute that aims to prevent and treat gambling addictions wants to build a dedicated mobile app to help lottery addicts better estimate their chances of winning. The institute has a team of engineers that will build the app, but they need me to create the logical core of the app and calculate probabilities.

For the first version of the app, they want me to focus on the [6/49 lottery](https://en.wikipedia.org/wiki/Lotto_6/49) and build functions that enable users to answer questions like:

- What is the probability of winning the big prize with a single ticket?
- What is the probability of winning the big prize if we play 40 different tickets (or any other number)?
- What is the probability of having at least five (or four, or three, or two) winning numbers on a single ticket?

The institute also wants me to consider historical data coming from the national 6/49 lottery game in Canada. The data set has data for 3,665 drawings, dating from 1982 to 2018.

The scenario I'm following throughout this project is fictional.

## Main functions

Throughout the project, I'll need to calculate repeatedly probabilities and combinations. As a consequence, I'll start by writing two functions that I'll use often:

- `factorial()`: a function that calculates factorials
- `combinations()`: a function that calculates combinations

Let's start coding the two functions.

In [0]:
# Factorial
def factorial(n):
  if (n == 0) or (n == 1):
      return 1
  elif n > 1:
      return n*factorial(n-1)
  else:
      return "ERROR - negative number"

# Combinations
def combinations(n, k):
  numerator = factorial(n)
  denominator = factorial(k) * factorial(n-k)
  return numerator/denominator

# Probability of Winning the Big Prize with a Single Ticket

Let's focus on writing a function `one_ticket_probability()` that calculates the probability of winning the big prize.

In the 6/49 lottery, six numbers are drawn from a set of 49 numbers that range from 1 to 49. A player wins the big prize if the six numbers on their tickets match all the six numbers drawn. If a player has a ticket with the numbers {13, 22, 24, 27, 42, 44}, he only wins the big prize if the numbers drawn are {13, 22, 24, 27, 42, 44}. If only one number differs, he doesn't win.

For the first version of the app, I want players to be able to calculate the probability of winning the big prize with the various numbers they play on a single ticket (for each ticket a player chooses six numbers out of 49). So, I'll start by building a function that calculates the probability of winning the big prize for any given ticket.

I discussed with the engineering team of the medical institute, and they told me that I need to be aware of the following details when I write the function:

- Inside the app, the user inputs six different numbers from 1 to 49.
- Under the hood, the six numbers will come as a Python list, which will serve as the single input to the function.
- The engineering team wants the function to print the probability value in a friendly way — in a way that people without any probability training are able to understand.


In [0]:
def one_ticket_probability(number_list):
  n_combinations = combinations(49, 6)
  percentage_prob = 100 / n_combinations

  print('''Your chances to win the big prize with the numbers {} are {:.6f}%. 
  In other words, you have a 1 in {:,} chances to win.'''.format(number_list, percentage_prob, int(n_combinations)))

In [44]:
test_input = [2, 43, 22, 23, 11, 5]
one_ticket_probability(test_input)

Your chances to win the big prize with the numbers [2, 43, 22, 23, 11, 5] are 0.000007%. 
  In other words, you have a 1 in 13,983,816 chances to win.


# Compare Ticket Against Historical Lottery Data


For the first version of the app, however, users should also be able to compare their ticket against the historical lottery data in Canada and determine whether they would have ever won by now.

Now I'll focus on exploring the historical data coming from the Canada 6/49 lottery. The data set can be downloaded from [Kaggle](https://www.kaggle.com/datascienceai/lottery-dataset) and it has the following structure:

In [45]:
# Import Libraries
import pandas as pd

lottery_canada = pd.read_csv('649.csv')
lottery_canada.shape

(3665, 11)

In [46]:
lottery_canada.head(3)

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
0,649,1,0,6/12/1982,3,11,12,14,41,43,13
1,649,2,0,6/19/1982,8,33,36,37,39,41,9
2,649,3,0,6/26/1982,1,6,23,24,27,39,34


The data set contains historical data for 3,665 drawings (each row shows data for a single drawing), dating from 1982 to 2018. For each drawing, there are the six numbers drawn in the following six columns:

- `NUMBER DRAWN 1`
- `NUMBER DRAWN 2`
- `NUMBER DRAWN 3`
- `NUMBER DRAWN 4`
- `NUMBER DRAWN 5`
- `NUMBER DRAWN 6`

## Create the Function for the Comparison

Now I'm going to write a function that will enable users to compare their ticket against the historical lottery data in Canada and determine whether they would have ever won by now.

The engineering team told me that I need to be aware of the following details:

- Inside the app, the user inputs six different numbers from 1 to 49.
- Under the hood, the six numbers will come as a Python list and serve as an input to the function.
- The engineering team wants me to write a function that prints:
    - the number of times the combination selected occurred in the Canada data set;
    - the probability of winning the big prize in the next drawing with that combination.

In [0]:
def count_historical_occurrence(number_list, historical_numbers):
  # generate a single column grouping all values in a set per every row
  single_column = pd.Series(historical_numbers.values.tolist()).apply(set)

  # check when there are matches
  matches = single_column == set(number_list)

  # count the matches
  matches_count = matches.sum()

  # output
  if matches_count == 0:
    print("The combination {} has never occured. This doesn't mean it's more likely to occur now.".format(number_list))
    one_ticket_probability(number_list)
        
  else:
    print("The number of times combination {} has occured in the past is {}.".format(number_list, matches_count))
    one_ticket_probability(number_list)


In [48]:
test_input = [33, 36, 37, 39, 8, 41]

count_historical_occurrence(test_input, lottery_canada.iloc[:, 4:10])

The number of times combination [33, 36, 37, 39, 8, 41] has occured in the past is 1.
Your chances to win the big prize with the numbers [33, 36, 37, 39, 8, 41] are 0.000007%. 
  In other words, you have a 1 in 13,983,816 chances to win.


# Probability of Winning the Big Prize with Multiple Tickets

Lottery addicts usually play more than one ticket on a single drawing, thinking that this might increase their chances of winning significantly. The goal here is to help them better estimate their chances of winning. Now, I'm going to write the function `multi_ticket_probability()` that will allow the users to calculate the chances of winning for any number of different tickets.

I've talked with the engineering team and they gave me the following information:

- The user will input the number of different tickets they want to play (without inputting the specific combinations they intend to play).
- The function will see an integer between 1 and 13,983,816 (the maximum number of different tickets).
- The function should print information about the probability of winning the big prize depending on the number of different tickets played.

Let's now start writing this function.

In [0]:
def multi_ticket_probability(n_tickets):
    
    n_combinations = combinations(49, 6)
    
    percentage_prob = 100 * n_tickets / n_combinations
    
    if n_tickets == 1:
        print('''Your chances to win the big prize with one ticket are {:.6f}%.
In other words, you have a 1 in {:,} chances to win.'''.format(percentage_prob, int(n_combinations)))
    
    else:
        print('''Your chances to win the big prize with {:,} different tickets are {:.6f}%.
In other words, you have a 1 in {:,} chances to win.'''.format(n_tickets, percentage_prob,
                                                               round(n_combinations / n_tickets)))

In [50]:
test_inputs = [1, 10, 100, 10000, 1000000, 6991908, 13983816]

for test_input in test_inputs:
    multi_ticket_probability(test_input)
    print('------------------------')

Your chances to win the big prize with one ticket are 0.000007%.
In other words, you have a 1 in 13,983,816 chances to win.
------------------------
Your chances to win the big prize with 10 different tickets are 0.000072%.
In other words, you have a 1 in 1,398,382 chances to win.
------------------------
Your chances to win the big prize with 100 different tickets are 0.000715%.
In other words, you have a 1 in 139,838 chances to win.
------------------------
Your chances to win the big prize with 10,000 different tickets are 0.071511%.
In other words, you have a 1 in 1,398 chances to win.
------------------------
Your chances to win the big prize with 1,000,000 different tickets are 7.151124%.
In other words, you have a 1 in 14 chances to win.
------------------------
Your chances to win the big prize with 6,991,908 different tickets are 50.000000%.
In other words, you have a 1 in 2 chances to win.
------------------------
Your chances to win the big prize with 13,983,816 different ti

# Probability of Winning with Less Numbers

Now I'm going to write one more function to allow the users to calculate probabilities for two, three, four, or five winning numbers.

In most 6/49 lotteries there are smaller prizes if a player's ticket match two, three, four, or five of the six numbers drawn. As a consequence, the users might be interested in knowing the probability of having two, three, four, or five winning numbers.

These are the engineering details I'll need to be aware of:

- Inside the app, the user inputs:
    - six different numbers from 1 to 49;
    - an integer between 2 and 5 that represents the number of winning numbers expected.
- Our function prints information about the probability of having the inputted number of winning numbers.

Let's write the function `one_ticket_probability_less_6()` that calculates the probability of having *exactly* the given number of winning numbers.

In [0]:
def one_ticket_probability_less_6(number_list, winning_n):
  # winning numbers over 6 elements
  winning_numbers = combinations(6, winning_n)

  #remaining numbers over 6 elements
  remaining_numbers = combinations(43, 6-winning_n)

  # total combinations
  total_combinations = combinations(49, 6)

  # final percentage probability
  percentage_prob = 100 * winning_numbers * remaining_numbers / total_combinations

  print('''Your chances of having {} winning numbers with this ticket are {:.6f}%.
In other words, you have a 1 in {:,} chances to win.'''.format(winning_n, percentage_prob, int(total_combinations/(winning_numbers*remaining_numbers))))

In [52]:
test_input = [33, 36, 37, 39, 8, 41]

for test_number in [2, 3, 4, 5]:
    one_ticket_probability_less_6(test_input, test_number)
    print('--------------------------')

Your chances of having 2 winning numbers with this ticket are 13.237803%.
In other words, you have a 1 in 7 chances to win.
--------------------------
Your chances of having 3 winning numbers with this ticket are 1.765040%.
In other words, you have a 1 in 56 chances to win.
--------------------------
Your chances of having 4 winning numbers with this ticket are 0.096862%.
In other words, you have a 1 in 1,032 chances to win.
--------------------------
Your chances of having 5 winning numbers with this ticket are 0.001845%.
In other words, you have a 1 in 54,200 chances to win.
--------------------------


# Conclusion

I have finally achieved the main goals in this project.

Possible features for a second version of the app include:

- Making the outputs even easier to understand by adding fun analogies (for example, I can find probabilities for strange events and compare with the chances of winning in lottery; for instance, I can output something along the lines "You are 100 times more likely to be the victim of a shark attack than winning the lottery")
- Combining the `one_ticket_probability()`  and `count_historical_occurrence()` to output information on probability and historical occurrence at the same time
- Create a function similar to `one_ticket_probability_less_6()` which calculates the probability of having _at least_ two, three, four or five winning numbers.